In [1]:
import sys
sys.path.append("../ARCH_package")

import basic

import dill
import pandas as pd
import os

import plotly.express as px
import plotly.graph_objects as go
colors = px.colors.qualitative.D3

In [2]:
# Import non-synonymous mutations as exported in LiFT.py
with open('../Exports/LBC_non-synonymous_LiFT_fitted.dill', 'rb') as infile:
    lbc_LiFT = dill.load(infile)

# Import non-synonymous mutations as exported in LiFT.py
with open('../Exports/LBC_non-synonymous_threshold_fitted.dill', 'rb') as infile:
    lbc_threshold = dill.load(infile)

In [3]:

LiFT_trajectories = [traj.fitness for part in lbc_LiFT for traj in part.trajectories]
threshold_trajectories = [traj.fitness for part in lbc_threshold for traj in part.trajectories]

LiFT_trajectory_ids = set([part.id + traj.mutation for part in lbc_LiFT for traj in part.trajectories])
threshold_trajectory_ids = set([part.id + traj.mutation for part in lbc_threshold for traj in part.trajectories])

LiFT_only = LiFT_trajectory_ids - threshold_trajectory_ids
LiFT_only_trajectories = [traj.fitness for part in lbc_LiFT for traj in part.trajectories
                          if part.id + traj.mutation in LiFT_only]
                          
threshold_only = threshold_trajectory_ids - LiFT_trajectory_ids
threshold_only_trajectories = [traj.fitness for part in lbc_threshold for traj in part.trajectories
                          if part.id + traj.mutation in threshold_only]

In [4]:
from scipy.stats import kruskal

kruskal(LiFT_trajectories, threshold_trajectories)

KruskalResult(statistic=14.362079605238801, pvalue=0.000150809024616589)

In [5]:
import plotly.graph_objects as go 

# Create path for exporting
path = f'../Results/LiFT/Comparison/'
if not os.path.exists(path):
    os.makedirs(path)


data_list = [LiFT_trajectories, threshold_trajectories, LiFT_only_trajectories, threshold_only_trajectories]
name_list = ['LiFT', '2%', 'LiFT only', '2% only']
color_list = [colors[0], colors[0], colors[0], colors[0]]
#color_list= ['Black', 'grey', 'grey', 'grey']

fig = go.Figure()
for data, name, color in zip(data_list, name_list, color_list):
    fig.add_trace(
        go.Box(
            y=data,
            name=name,
            boxpoints='all',
            marker_color = color,
            showlegend=False
        )
    )
fig.update_layout(yaxis_title='Fitness')

fig.show()
fig.write_image(path + 'LiFT_vs_threshold_fitness.png', scale=10)
fig.write_image(path + 'LiFT_vs_threshold_fitness.svg')